In [6]:
import sys
import json
from pycarol.auth.PwdAuth import PwdAuth
from pycarol import Carol, ApiKeyAuth, Subscription
from pycarol.staging import Staging
from pycarol.data_models import DataModel
from pycarol.tenant import Tenant
from pycarol import Storage, Connectors
import pandas as pd
import os
from collections import defaultdict
from dotenv import load_dotenv
load_dotenv('.env', override=True)
import time
from pycarol.apps import Apps
import pycarol
from functools import reduce

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.min_rows', 100)


domain = 'protheusdev'
env = 'carol.ai'
org = 'totvstechfindev'

email = os.environ['CAROLUSER']
password = os.environ['CAROLPWD']

login = Carol(domain, 'clockinapp', auth=PwdAuth(email, password), environment=env, organization=org, )
carol_app = Apps(login).all()
if carol_app:
    carol_app, _ = carol_app.popitem()
else:
    print("no carol app")
    carol_app = "aaa"
login.app_name = carol_app

stag  = Staging(login)

print(pycarol.__version__)


2.43.0


In [8]:
import gspread

gc = gspread.service_account(filename='cred.json')

sh = gc.open("status_techfin_reprocess")

print(sh.sheet1.get('A1'))

[['Código T']]


In [ ]:
from functions import carol_task
import time
while True:
    pross_tasks = carol_task.find_task_types(login)
    pross_task = [i['mdmId'] for i in pross_tasks]
    if pross_task:
        carol_task.cancel_tasks(login, pross_task)
    time.sleep(4)

In [11]:
def cancel(login):
    while True:
        pross_tasks = carol_task.find_task_types(login)
        pross_task = [i['mdmId'] for i in pross_tasks]
        if pross_task:
            carol_task.cancel_tasks(login, pross_task)
        time.sleep(4)

In [14]:
import multiprocessing
proc = multiprocessing.Process(target=cancel, args=(login,))

proc.start()

In [3]:
from functions import custom_pipeline
dag = custom_pipeline.get_dag()

In [7]:
dag = list(reduce(set.union, custom_pipeline.get_dag()))

In [9]:
dms = [i.replace('DM_', '') for i in dag if i.startswith('DM_')]

In [14]:
dms

['apinvoiceaccounting',
 'apinvoicepayments',
 'apinvoice',
 'arinvoice',
 'arinvoiceinstallment',
 'arinvoicebra',
 'arinvoiceaccounting',
 'apinvoiceinstallment',
 'arinvoicepayments',
 'apinvoicebra',
 'arinvoiceorigin',
 'arinvoicepartner']

# Terminate the process
proc.terminate()  # sends a SIGTERM
proc.exitcode

In [3]:
org = 'totvstechfin'
app_name = "techfinplatform"
app_version = '0.0.65'
connector_name = 'protheus_carol'
connector_group = 'protheus'

consolidate_list = ['se1', 'se2', ]

In [26]:
from functions import sheet_utils, carol_login, carol_apps, carol_task, custom_pipeline, techfin_task
from functools import reduce

In [5]:
dag = custom_pipeline.get_dag()
dag = list(reduce(set.union, custom_pipeline.get_dag()))
dms = [i.replace('DM_', '') for i in dag if i.startswith('DM_')]
staging_list = [i for i in dag if not i.startswith('DM_')]

In [6]:
import logging
logger = logging.getLogger('app')

In [15]:
task_list = carol_task.par_delete_golden(login, dm_list=dms, n_jobs=1)

Deleted:  {'count': 2067}
Deleted:  {'count': 0}
Deleted:  {'count': 895}
Deleted:  {'count': 0}
Deleted:  {'count': 663}
Deleted:  {'count': 0}
Deleted:  {'count': 2165}
Deleted:  {'count': 15}
Deleted:  {'count': 826}
Deleted:  {'count': 0}
Deleted:  {'count': 1}
Deleted:  {'count': 0}


In [ ]:
st = carol_task.get_all_stagings(login, connector_name=connector_name)
st = [i for i in st if i.startswith('se1_') or i.startswith('se2_')]

In [7]:
st

['se1_acresc',
 'se1_decresc',
 'se1_installments',
 'se1_invoice',
 'se1_payments',
 'se1_payments_abatimentos',
 'se2_acresc',
 'se2_decresc',
 'se2_installments',
 'se2_invoice',
 'se2_payments',
 'se2_payments_abatimentos']

In [8]:
task_list = carol_task.par_delete_staging(login, staging_list=st, connector_name=connector_name, n_jobs=1)

In [10]:
task_list

['794dc87001094cee91bcd7b9dec794c8',
 '4758aa8e39074b7090c52bb461fb7a9b',
 '6b4697a290274107bcc84137e0233e3b',
 '0b7ec9e7236749b7a0ccd390ea2feef7',
 '363328bdcf3442f8b122d891798d0390',
 'd53214c78ea9473481e9906781bb513d',
 '8e0abd2751f34d97a4b220e57063732f',
 '8e5977a89dba425caa3d4739771d1b09',
 '9cc9dbc90f4048eb8fb058e46f4d48de',
 'b243352d22f14f16a847e1d8a2957324',
 'd26a1d17e8644c0492273147e9a45402',
 '8564ce2d5826414597813d8fd5dbd900']

In [18]:
connector_name='protheus_carol'
conn_stats = Connectors(login).stats(connector_name=connector_name)
st = [i for i in list(conn_stats.values())[0]]
st = sorted(st)

In [28]:
[i for i in st if (i.startswith('se2_')  )]

['se2_acresc',
 'se2_decresc',
 'se2_installments',
 'se2_invoice',
 'se2_payments',
 'se2_payments_abatimentos']

In [29]:
['sf1_consulta', 'sf1_invoicebra', 'sf2_consulta', 'sf2_invoicebra'] + \
              ['sd1_consulta', 'sd1_dados', 'sd1_devolution', 'sd1_else'] + \
                [
                'se1_installments',
                'se1_invoice',
                'se1_payments',
                'se1_payments_abatimentos'] + [
 'se2_installments',
 'se2_invoice',
 'se2_payments',
 'se2_payments_abatimentos']

['sf1_consulta',
 'sf1_invoicebra',
 'sf2_consulta',
 'sf2_invoicebra',
 'sd1_consulta',
 'sd1_dados',
 'sd1_devolution',
 'sd1_else',
 'se1_installments',
 'se1_invoice',
 'se1_payments',
 'se1_payments_abatimentos',
 'se2_installments',
 'se2_invoice',
 'se2_payments',
 'se2_payments_abatimentos']

In [16]:
from pycarol import CDSStaging

In [84]:
res = {}
for staging in sts:
    key = staging+'_'+field
    mapping_id = conn.get_dm_mappings(connector_name='protheus_carol', staging_name=staging, )[0]['mdmId']
    mappings_to_get = stag.get_mapping_snapshot(connector_id=connector_id, mapping_id=mapping_id, )
    check = [i for i in mappings_to_get.get(None)['fieldMappings'] if i['targetFieldName']==field]
    if check: 
        for sub_mapping in check:
            if len([i for i in find_keys(sub_mapping, 'mdmName') if i=='MD5_HASH'])==0:
                continue
            else:
                sub_mapping = sub_mapping['fieldCleanseRules']
            #sanity_check (i dont know if this covvers nested functions)
            assert(len([i for i in find_keys(sub_mapping, 'mdmName') if i=='MD5_HASH'])==1)
            unest_func = list(chain(*(list(find_keys(sub_mapping, 'actions'))
                                                    + list(find_keys(sub_mapping, 'falseActions'))
                                                    + [sub_mapping, ])))
            res[key] = [i['parameterValues'] for i in unest_func if i['fieldFunction']['mdmName']=='MD5_HASH']
    else:
        res[key] = None